# LSTM을 이용한 텍스트 생성
- 데이터 다운로드: 캐글 New York Times Comments
- ArticlesApril2018.csv

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

### 파일 업로드

In [3]:
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving ArticlesApril2018.csv to ArticlesApril2018.csv


In [4]:
import pandas as pd

df = pd.read_csv(filename, encoding='latin1')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleadersâ Settlement Offer...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"âI understand that they could meet with us, ...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,Whatâs it like to eat at the second incarnat...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Torontoâs ...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


### 데이터 전처리

In [5]:
print('열의 개수: ', len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [6]:
df['headline'].isnull().values.any() # headline 열의 데이터만 사용하기 때문에 null값 있는지 확인

False

In [7]:
headline = [text for text in df.headline.values]
headline[:5]

['Former N.F.L. Cheerleadersâ\x80\x99 Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [8]:
len(headline)

1324

In [9]:
headline = [text for text in headline if text != 'Unknown']
len(headline)

1214

In [10]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [11]:
def repreprocessing(s):     # 구두점 제거
  s = s.encode('utf8').decode('ascii', 'ignore')
  return ''.join(c for c in s if c not in punctuation).lower()

In [12]:
text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [13]:
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기: %d' % vocab_size)

단어 집합의 크기: 3494


In [14]:
sequences = []
for line in text:
  encoded = t.texts_to_sequences([line])[0]
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)
sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [15]:
index_to_word = {}
for key, value in t.word_index.items():
  index_to_word[value] = key

print('빈도수 상위 1번 단어: ', index_to_word[1])
print('빈도수 상위 582번 단어: ', index_to_word[582])

빈도수 상위 1번 단어:  the
빈도수 상위 582번 단어:  offer


In [16]:
max_len = max(len(s) for s in sequences)
print('샘플의 최대 길이: ', max_len)

샘플의 최대 길이:  24


In [17]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [18]:
X = sequences[:, :-1]
y = sequences[:, -1]

Y = to_categorical(y, num_classes=vocab_size)

In [19]:
X.shape, Y.shape

((7803, 23), (7803, 3494))

### 모델 정의/설정/학습

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GRU

In [21]:
model = Sequential([
                    Embedding(vocab_size, 10, input_length=max_len - 1, name='Embedding_Layer'),
                    GRU(128, name='GRU_Layer'),
                    Dense(vocab_size, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, 23, 10)            34940     
_________________________________________________________________
GRU_Layer (GRU)              (None, 128)               53760     
_________________________________________________________________
dense (Dense)                (None, 3494)              450726    
Total params: 539,426
Trainable params: 539,426
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
history = model.fit(X, Y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 10s - loss: 7.7057 - accuracy: 0.0291
Epoch 2/200
244/244 - 3s - loss: 7.1229 - accuracy: 0.0315
Epoch 3/200
244/244 - 3s - loss: 6.9182 - accuracy: 0.0432
Epoch 4/200
244/244 - 3s - loss: 6.7109 - accuracy: 0.0495
Epoch 5/200
244/244 - 3s - loss: 6.4869 - accuracy: 0.0534
Epoch 6/200
244/244 - 3s - loss: 6.2371 - accuracy: 0.0556
Epoch 7/200
244/244 - 3s - loss: 5.9657 - accuracy: 0.0624
Epoch 8/200
244/244 - 3s - loss: 5.6905 - accuracy: 0.0679
Epoch 9/200
244/244 - 3s - loss: 5.4227 - accuracy: 0.0759
Epoch 10/200
244/244 - 3s - loss: 5.1652 - accuracy: 0.0834
Epoch 11/200
244/244 - 3s - loss: 4.9260 - accuracy: 0.0951
Epoch 12/200
244/244 - 3s - loss: 4.6989 - accuracy: 0.1153
Epoch 13/200
244/244 - 3s - loss: 4.4823 - accuracy: 0.1402
Epoch 14/200
244/244 - 3s - loss: 4.2784 - accuracy: 0.1654
Epoch 15/200
244/244 - 3s - loss: 4.0816 - accuracy: 0.2011
Epoch 16/200
244/244 - 3s - loss: 3.8975 - accuracy: 0.2300
Epoch 17/200
244/244 - 3s - loss: 3.7178 - accur

### 모델 검증

In [24]:
def sentence_generation(model, t, current_word, n):
  init_word = current_word
  sentence = ''
  for _ in range(n):
    encoded = t.texts_to_sequences([current_word])[0]
    encoded = pad_sequences([encoded], maxlen=23, padding='pre')
    result = np.argmax(model.predict(encoded), axis=-1)
    for word, index in t.word_index.items():
      if index == result:
        break
    current_word = current_word + ' ' + word
    sentence = sentence + ' ' + word
    
  sentence = init_word + sentence
  return sentence

In [25]:
print(sentence_generation(model, t, 'i', 10))
print(sentence_generation(model, t, 'how', 10))
print(sentence_generation(model, t, 'former', 10))

i want to be rich and im not sorry scientists beg
how to make a crossword puzzle vote children currency be forgotten
former nfl cheerleaders settlement offer 1 and a meeting with goodell
